In [1]:
from model import get_model

/gpfs/home/mathek07/.cache/pypoetry/virtualenvs/prostate-mri-classification-VcvKyKw0-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# !jupyter kernelspec remove -y intelligent-scanning-310

Removed /gpfs/home/mathek07/.local/share/jupyter/kernels/intelligent-scanning-310


In [2]:

import yaml

with open("config/resnet18.yaml", 'r') as f:
    config = yaml.safe_load(f)

In [3]:
model = get_model(config)
model

/gpfs/home/mathek07/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/gpfs/home/mathek07/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


MRIResNet3D(
  (axt2_resnet): VideoResNet(
    (stem): BasicStem(
      (0): Conv3d(1, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
      (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Sequential(
          (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
          (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (conv2): Sequential(
          (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
          (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (relu): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Sequential(
          (0): Conv3DSimple(64, 64, kernel_size=

In [7]:
import torch

input = {
    "axt2": torch.randn(4, 1, 30, 180, 180),
    "adc": torch.randn(4, 1, 22, 130, 130),
    "b1500": torch.randn(4, 1, 22, 130, 130),
}
output = model(input)
print(output)
output.size()

tensor([[-0.4340,  0.0531,  0.1589, -0.0919,  0.0358],
        [-0.0011,  0.1162,  0.1553, -0.0349,  0.0569],
        [-0.2391,  0.2990,  0.4061,  0.0208, -0.1107],
        [-0.2354,  0.0601,  0.1637, -0.0950,  0.0102]],
       grad_fn=<AddmmBackward0>)


torch.Size([4, 5])

In [8]:
import pandas as pd

# Read the CSV file
train_csv_path = config['paths']['train_csv']
train_df = pd.read_csv(train_csv_path)

# Display the head of the dataframe
train_df.head()

,PatientID,AccessionNumber,StudyDate,maxPIRADS
0,2071143.0,9768559,20150226.0,3.0
1,534143.0,9815271,20150303.0,1.0
2,576605.0,9827480,20150304.0,4.0
3,34464.0,9810380,20150305.0,4.0
4,402433.0,9843288,20150327.0,1.0


In [20]:
import pandas as pd
import torch

def calculate_class_weights(config, csv_path):
    num_classes = config["train"]["model"]["num_classes"]
    df = pd.read_csv(csv_path)
    if config["train"]["binarize"]:
        labels = (df["maxPIRADS"] > 2).astype(int)  # Binarize: 1 and 2 -> 0, >2 -> 1
    else:
        labels = df["maxPIRADS"].astype(int) - 1  # Adjust labels to be 0-4
    label_counts = labels.value_counts().sort_index()
    total_samples = len(labels)
    class_weights = total_samples / (num_classes * label_counts)
    return torch.tensor(class_weights.values, dtype=torch.float)

config = {
    "data": {
        "series": ["axt2", "adc", "b1500"],
        "val_split": 0.2,
        "batch_size": 16,  # 128
        "num_workers": 4,
        "augment_data": False,
        "normalize": True,
        "zero_pad": True
    },
    "paths": {
        "data_dir": "/gpfs/data/prostatelab/processed_data/h5/t2_dwi_filtered_20240212",
        "train_csv": "/gpfs/data/prostatelab/processed_data/csv/data_split/train_temporal_split_20240619_exclude_mismatch.csv",
        "valid_csv": "/gpfs/data/prostatelab/processed_data/csv/data_split/val_temporal_split_20240619_exclude_mismatch.csv"
    },
    "train": {
        "epochs": 30,
        "model": {
            "name": "MRI_ResNet50_3D",
            "pretrained": False,
            "num_classes": 2,
            "combine_adc_b1500": True
        },
        "binarize": True,
        "criterion": "ce_weighted",  # Loss function: "ce", "mse", or "bce"
        "optimizer": {
            "name": "adamw",  # Optimizer: "adamw", "adam", "sgd", or "rmsprop"
            "lr": 0.0005
        },
        "scheduler": {
            "name": "cosine"  # Scheduler: "steplr", "exponentiallr", "reducelronplateau", "cosine", "onecycle", or "linear"
        }
    },
    "logging": {
        "wandb_project": "Prostate MRI Classification",
        "wandb_log_files": ["train.log", "valid.log"]
    }
}

calculate_class_weights(config, config["paths"]["train_csv"])

tensor([0.9413, 1.0665])

In [17]:
df = pd.read_csv(config["paths"]["train_csv"])
df["maxPIRADS"] = (df["maxPIRADS"] > 2)
df["maxPIRADS"].value_counts().sort_index()

maxPIRADS
False    13880
True     12251
Name: count, dtype: int64

In [21]:
from loader.data import get_loaders

train_loader, val_loader = get_loaders(config)

Global batch size: 16
Using SequentialSampler for train_loader
Local batch size per GPU: 16


In [22]:
for a in train_loader:
    print(a)
    break

{'vol': {'axt2': tensor([[[[[-1.2396e+00, -1.2054e+00, -1.1370e+00,  ..., -1.3275e+00,
            -1.3324e+00, -1.3617e+00],
           [-1.1907e+00, -1.2493e+00, -1.2445e+00,  ..., -1.3324e+00,
            -1.3812e+00, -1.3665e+00],
           [-1.1565e+00, -1.2933e+00, -1.3226e+00,  ..., -1.3079e+00,
            -1.3714e+00, -1.3959e+00],
           ...,
           [ 9.3697e-02,  5.4627e-02,  6.4394e-02,  ..., -9.2701e-01,
            -6.9259e-01, -6.8771e-01],
           [ 8.3929e-02,  1.1323e-01,  8.8813e-02,  ..., -1.1907e+00,
            -9.6608e-01, -8.6841e-01],
           [ 1.5556e-02, -1.8630e-02, -1.3746e-02,  ..., -8.2445e-01,
            -7.6585e-01, -9.2213e-01]],

          [[-1.2396e+00, -1.1712e+00, -1.2493e+00,  ..., -1.3617e+00,
            -1.3568e+00, -1.4056e+00],
           [-1.2640e+00, -1.1956e+00, -1.1810e+00,  ..., -1.3617e+00,
            -1.3128e+00, -1.3617e+00],
           [-1.2396e+00, -1.2298e+00, -1.1272e+00,  ..., -1.3275e+00,
            -1.3763e+00

In [10]:
from monai.networks.nets import SEResNext50

model = SEResNext50(
            spatial_dims=3,
            in_channels=1,  # Single-channel input for AXT2
            num_classes=0,  # Explicitly set to 0 to avoid classification layer initialization
            pretrained=False
        )

model

SEResNext50(
  (layer0): Sequential(
    (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (pool): MaxPool3d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  )
  (layer1): Sequential(
    (0): SEResNeXtBottleneck(
      (conv1): Convolution(
        (conv): Conv3d(64, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (adn): ADN(
          (N): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (A): ReLU(inplace=True)
        )
      )
      (conv2): Convolution(
        (conv): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), groups=32, bias=False)
        (adn): ADN(
          (N): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (A): ReLU(inplace=True)
        )
      )
    

In [11]:
import gc
import yaml

def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

with open("config/resnet18.yaml", 'r') as f:
    resnet18_config = yaml.safe_load(f)

with open("config/resnet50.yaml", 'r') as f:
    resnet50_config = yaml.safe_load(f)

with open("config/seresnext50.yaml", 'r') as f:
    seresnext50_config = yaml.safe_load(f)

model = get_model(resnet18_config)
print(f"ResNet18: {count_params(model)} trainable parameters")
del model
gc.collect()

model = get_model(resnet50_config)
print(f"ResNet50: {count_params(model)} trainable parameters")
del model
gc.collect()

import torch
import torch.nn as nn
from monai.networks.nets import SEResNext50

class MRI_SEResNext50_3D(nn.Module):
    def __init__(self, config):
        super(MRI_SEResNext50_3D, self).__init__()
        self.config = config

        # AXT2 network
        self.axt2_seresnext = SEResNext50(
            spatial_dims=3,
            in_channels=1,  # Single-channel input for AXT2
            num_classes=0,  # Set to None to use as feature extractor
            pretrained=False  # Pretraining not supported for 3D
        )
        self.axt2_seresnext.last_linear = nn.Identity()  # Remove classification layer

        # ADC and B1500 network configuration
        if config["train"]["model"]["combine_adc_b1500"]:
            self.adc_b1500_seresnext = SEResNext50(
                spatial_dims=3,
                in_channels=2,  # Combine ADC and B1500 as channels
                num_classes=0,
                pretrained=False
            )
            self.adc_b1500_seresnext.last_linear = nn.Identity()
        else:
            self.adc_seresnext = SEResNext50(
                spatial_dims=3,
                in_channels=1,
                num_classes=0,
                pretrained=False
            )
            self.adc_seresnext.last_linear = nn.Identity()

            self.b1500_seresnext = SEResNext50(
                spatial_dims=3,
                in_channels=1,
                num_classes=0,
                pretrained=False
            )
            self.b1500_seresnext.last_linear = nn.Identity()

        # Fully connected layer for classification
        feature_dim = 2048  # SEResNext50 output size
        num_inputs = feature_dim * 2 if config["train"]["model"]["combine_adc_b1500"] else feature_dim * 3
        self.fc = nn.Sequential(
            nn.Linear(num_inputs, 256),
            nn.ReLU(),
            # nn.Dropout(0.3),
            nn.Linear(256, config["train"]["model"]["num_classes"])
        )

    def forward(self, x):
        axt2 = x['axt2']
        adc = x['adc']
        b1500 = x['b1500']

        # Extract features for AXT2
        axt2_features = self.axt2_seresnext(axt2)
        axt2_features = axt2_features.view(axt2_features.size(0), -1)

        # Combine ADC and B1500 features
        if self.config["train"]["model"]["combine_adc_b1500"]:
            adc_b1500 = torch.cat([adc, b1500], dim=1)  # Combine as channels
            adc_b1500_features = self.adc_b1500_seresnext(adc_b1500)
            combined_features = torch.cat([axt2_features, adc_b1500_features], dim=1)
        else:
            adc_features = self.adc_seresnext(adc)
            b1500_features = self.b1500_seresnext(b1500)
            combined_features = torch.cat([axt2_features, adc_features, b1500_features], dim=1)

        # Classification layer
        output = self.fc(combined_features)
        return output


model = MRI_SEResNext50_3D(seresnext50_config)
print(f"SEResNext50: {count_params(model)} trainable parameters")
del model
gc.collect()

ResNet18: 66568005 trainable parameters
ResNet50: 93389122 trainable parameters
SEResNext50: 57776802 trainable parameters


0

In [14]:
model = get_model(resnet50_config)

# model = MRI_SEResNext50_3D(seresnext50_config)

print(model)
# del model
# gc.collect()

MRI_ResNet50_3D(
  (axt2_resnet): ResNet(
    (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(1, 1, 1), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU(inplace=True)
    (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): ResNetBottleneck(
        (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn3): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU(inplace=True

In [15]:
from monai.networks.nets.resnet import ResNet, ResNetBottleneck, get_inplanes
from monai.networks.nets import SEResNext50

model1 = SEResNext50(
                spatial_dims=3,
                in_channels=2,  # Combine ADC and B1500 as channels
                num_classes=0,
                pretrained=False
            )
model1.last_linear = nn.Identity()

block = ResNetBottleneck
layers = [3, 4, 6, 3]  # ResNet50 configuration
block_inplanes = get_inplanes()

model2 = ResNet(
            spatial_dims=3,
            n_input_channels=1,  # Single-channel input for AXT2
            block=block,
            layers=layers,
            block_inplanes=block_inplanes,
            num_classes=0  # Set to 0 to use as feature extractor
        )
model2.fc = nn.Identity()

count_params(model1), count_params(model2)

/gpfs/home/mathek07/.cache/pypoetry/virtualenvs/prostate-mri-classification-VcvKyKw0-py3.12/lib/python3.12/site-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


(28374704, 46158912)

In [17]:
model2

ResNet(
  (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(1, 1, 1), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): ReLU(inplace=True)
  (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResNetBottleneck(
      (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn3): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv3d(64, 25

In [19]:
!poetry add einops

The following packages are already present in the pyproject.toml and will be skipped:

  - einops

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


In [18]:
from monai.networks.nets import ViT
from utils import get_series_properties

def init_vit(series_key, in_channels):
    props = get_series_properties(series_key)
    return ViT(
        in_channels=in_channels,
        img_size=(props["crop_size"][0], props["crop_size"][1], props["num_slices"]),
        patch_size=(16, 16, 8),  # Adjust as per dataset requirements
        hidden_size=768,
        mlp_dim=3072,
        num_layers=12,
        num_heads=12,
        spatial_dims=3,
        classification=False
    )

model = init_vit("axt2", 1)
model

OptionalImportError: from einops.layers.torch import Rearrange (No module named 'einops').

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies

In [24]:
from monai.networks.nets import ViT
from utils import get_series_properties

series_key = "axt2"
props = get_series_properties(series_key)

patch_size = {
    "axt2": (4, 16, 16),
    "adc": (2, 26, 26),
    "b1500": (2, 26, 26)
}[series_key]
model = ViT(
    in_channels=1,
    img_size=(props["num_slices"], props["crop_size"][0], props["crop_size"][1]),
    patch_size=patch_size,
    hidden_size=768,
    mlp_dim=3072,
    num_layers=12,
    num_heads=12,
    spatial_dims=3,
    classification=False
)

print(f"image size: {(props["num_slices"], props["crop_size"][0], props["crop_size"][1])}, patch size: {model.patch_embedding.patch_embeddings.stride}")

image size: (30, 180, 180), patch size: (4, 16, 16)


In [40]:
import torch

inp = torch.randn(4, 1, 30, 180, 180)
a, b = model(inp)

print(a.size())
print(len(b))

for c in b:
    print(c.size())

torch.Size([4, 847, 768])
12
torch.Size([4, 847, 768])
torch.Size([4, 847, 768])
torch.Size([4, 847, 768])
torch.Size([4, 847, 768])
torch.Size([4, 847, 768])
torch.Size([4, 847, 768])
torch.Size([4, 847, 768])
torch.Size([4, 847, 768])
torch.Size([4, 847, 768])
torch.Size([4, 847, 768])
torch.Size([4, 847, 768])
torch.Size([4, 847, 768])


In [36]:
print(a.size())
print(a[-1].size())
print(a[-1].mean(dim=1).size())

print(type(a))
print(a.size())
# Correctly access the last layer output
last_layer_output = a[-1]  # Output of the final layer

# Shape: [batch_size, num_patches, hidden_size], e.g., [1, 847, 768]

# Perform mean pooling across patches (dim=1)
pooled_features = last_layer_output.mean(dim=1)

print(last_layer_output.size())  # Correctly outputs: torch.Size([1, 847, 768])
print(pooled_features.size())   # Correctly outputs: torch.Size([1, 768])

torch.Size([1, 847, 768])
torch.Size([847, 768])
torch.Size([847])
<class 'torch.Tensor'>
torch.Size([1, 847, 768])
torch.Size([847, 768])
torch.Size([847])
